In [1]:
import pathlib
import chess
import chess.pgn
import chess.engine
import numpy as np
import h5py

In [2]:
pgnGames = list(pathlib.Path('').glob('*.pgn'))

In [3]:
# General space saving algorithms that are often used in chess programming sites

def board_to_arr(board: chess.Board):
    black, white = board.occupied_co

    bitboards = np.array([
        black & board.pawns,
        black & board.knights,
        black & board.bishops,
        black & board.rooks,
        black & board.queens,
        black & board.kings,
        white & board.pawns,
        white & board.knights,
        white & board.bishops,
        white & board.rooks,
        white & board.queens,
        white & board.kings,
    ], dtype=np.uint64)

    return bitboards

def bitboards_to_array(bb: np.ndarray) -> np.ndarray:
    bb = np.asarray(bb, dtype=np.uint64)[:, np.newaxis]
    s = 8 * np.arange(7, -1, -1, dtype=np.uint64)
    b = (bb >> s).astype(np.uint8)
    b = np.unpackbits(b, bitorder="little")
    return b.reshape(-1, 8, 8)


In [4]:
engine = chess.engine.SimpleEngine.popen_uci("engines/stockfish-macos-x86-64-bmi2")

def convert_to_winrate(acpl):
    
    return (50 + 50 * (2 / (1 + np.exp(-0.00368208 * acpl)) - 1)) * 0.01

def evaluate_game(game, think_time=0.1, verbose=False):
    
    single_game = []
    evaluations = []
    
    board = game.board()

    for move in game.mainline_moves():
    
        # Evaluate the current position 
        result = engine.analyse(board, chess.engine.Limit(time=think_time))
        
        if result['score'].turn: 
            try:
                objective_eval = result['score'].relative.cp
                evaluations.append(convert_to_winrate(objective_eval))
            except:
                if result['score'].relative.moves > 0:
                    evaluations.append(1)
                else:
                    evaluations.append(0)
        else:
            try:
                objective_eval = result['score'].relative.cp * -1
                evaluations.append(convert_to_winrate(objective_eval))
            except:
                if result['score'].relative.moves < 0:
                    evaluations.append(1)
                else:
                    evaluations.append(0)       
        
        # Push the game state to new
        board.push_uci(move.uci())
        single_game.append(board_to_arr(board))
    
    # Evaluate final position once more to find accuracy of last move
    result = engine.analyse(board, chess.engine.Limit(time=think_time))
    evaluations.append(convert_to_winrate(objective_eval))
    if verbose:
        print(evaluations)
    
    winrate_diff = [abs(x - evaluations[i - 1]) for i, x in enumerate(evaluations)][1:]

    return single_game, winrate_diff
    

### Generate Dataset

In [5]:
X = []
y = []

for g in range(len(pgnGames)):
    pgn = open(pgnGames[g])
    
    for k in range(25000):  # 190,000 assures all games are looked at.
        
        try:
            game = chess.pgn.read_game(pgn)

            board_history, winrate_history = evaluate_game(game, think_time=0.025, verbose=False)
            X += board_history
            y += winrate_history
            print(f"{k+1} games analyzed, dataset size {len(X)}")
        except Exception as e:
            print(f"{e}")
            pass
          

1 games analyzed, dataset size 55
2 games analyzed, dataset size 192
3 games analyzed, dataset size 251
4 games analyzed, dataset size 383
5 games analyzed, dataset size 481
6 games analyzed, dataset size 572
7 games analyzed, dataset size 625
8 games analyzed, dataset size 670
9 games analyzed, dataset size 695
10 games analyzed, dataset size 816
11 games analyzed, dataset size 869
12 games analyzed, dataset size 991
13 games analyzed, dataset size 1059
14 games analyzed, dataset size 1115
15 games analyzed, dataset size 1210
16 games analyzed, dataset size 1302
17 games analyzed, dataset size 1372
18 games analyzed, dataset size 1432
19 games analyzed, dataset size 1507
20 games analyzed, dataset size 1583
21 games analyzed, dataset size 1663
22 games analyzed, dataset size 1707
23 games analyzed, dataset size 1792
24 games analyzed, dataset size 1949
25 games analyzed, dataset size 2001
26 games analyzed, dataset size 2069
27 games analyzed, dataset size 2136
28 games analyzed, data

218 games analyzed, dataset size 18287
219 games analyzed, dataset size 18362
220 games analyzed, dataset size 18448
221 games analyzed, dataset size 18520
222 games analyzed, dataset size 18578
223 games analyzed, dataset size 18683
224 games analyzed, dataset size 18735
225 games analyzed, dataset size 18836
226 games analyzed, dataset size 18933
227 games analyzed, dataset size 18992
228 games analyzed, dataset size 19062
229 games analyzed, dataset size 19115
230 games analyzed, dataset size 19168
231 games analyzed, dataset size 19426
232 games analyzed, dataset size 19536
233 games analyzed, dataset size 19642
234 games analyzed, dataset size 19705
235 games analyzed, dataset size 19785
236 games analyzed, dataset size 19877
237 games analyzed, dataset size 19936
238 games analyzed, dataset size 19970
239 games analyzed, dataset size 20058
240 games analyzed, dataset size 20128
241 games analyzed, dataset size 20184
242 games analyzed, dataset size 20241
243 games analyzed, datas

429 games analyzed, dataset size 36170
430 games analyzed, dataset size 36251
431 games analyzed, dataset size 36336
432 games analyzed, dataset size 36385
433 games analyzed, dataset size 36542
434 games analyzed, dataset size 36586
435 games analyzed, dataset size 36678
436 games analyzed, dataset size 36747
437 games analyzed, dataset size 36844
438 games analyzed, dataset size 36932
439 games analyzed, dataset size 37016
440 games analyzed, dataset size 37099
441 games analyzed, dataset size 37121
442 games analyzed, dataset size 37258
443 games analyzed, dataset size 37318
444 games analyzed, dataset size 37375
445 games analyzed, dataset size 37468
446 games analyzed, dataset size 37580
447 games analyzed, dataset size 37719
448 games analyzed, dataset size 37771
449 games analyzed, dataset size 37876
450 games analyzed, dataset size 37971
451 games analyzed, dataset size 38121
452 games analyzed, dataset size 38235
453 games analyzed, dataset size 38291
454 games analyzed, datas

640 games analyzed, dataset size 53150
641 games analyzed, dataset size 53215
642 games analyzed, dataset size 53299
643 games analyzed, dataset size 53372
644 games analyzed, dataset size 53422
645 games analyzed, dataset size 53513
646 games analyzed, dataset size 53551
647 games analyzed, dataset size 53673
648 games analyzed, dataset size 53721
649 games analyzed, dataset size 53877
650 games analyzed, dataset size 53982
651 games analyzed, dataset size 54047
652 games analyzed, dataset size 54061
653 games analyzed, dataset size 54212
654 games analyzed, dataset size 54282
655 games analyzed, dataset size 54380
656 games analyzed, dataset size 54497
657 games analyzed, dataset size 54577
658 games analyzed, dataset size 54647
659 games analyzed, dataset size 54722
660 games analyzed, dataset size 54841
661 games analyzed, dataset size 54953
662 games analyzed, dataset size 55115
663 games analyzed, dataset size 55248
664 games analyzed, dataset size 55352
665 games analyzed, datas

851 games analyzed, dataset size 71901
852 games analyzed, dataset size 72068
853 games analyzed, dataset size 72112
854 games analyzed, dataset size 72171
855 games analyzed, dataset size 72264
856 games analyzed, dataset size 72404
857 games analyzed, dataset size 72518
858 games analyzed, dataset size 72575
859 games analyzed, dataset size 72600
860 games analyzed, dataset size 72669
861 games analyzed, dataset size 72740
862 games analyzed, dataset size 72864
863 games analyzed, dataset size 72948
864 games analyzed, dataset size 73024
865 games analyzed, dataset size 73109
866 games analyzed, dataset size 73157
867 games analyzed, dataset size 73274
868 games analyzed, dataset size 73401
869 games analyzed, dataset size 73517
870 games analyzed, dataset size 73585
871 games analyzed, dataset size 73617
872 games analyzed, dataset size 73691
873 games analyzed, dataset size 73806
874 games analyzed, dataset size 73891
875 games analyzed, dataset size 73954
876 games analyzed, datas

...

24057 games analyzed, dataset size 1980716
24058 games analyzed, dataset size 1980864
24059 games analyzed, dataset size 1980910
24060 games analyzed, dataset size 1981006
24061 games analyzed, dataset size 1981104
24062 games analyzed, dataset size 1981184
24063 games analyzed, dataset size 1981259
24064 games analyzed, dataset size 1981366
24065 games analyzed, dataset size 1981490
24066 games analyzed, dataset size 1981579
24067 games analyzed, dataset size 1981659
24068 games analyzed, dataset size 1981763
24069 games analyzed, dataset size 1981881
24070 games analyzed, dataset size 1982012
24071 games analyzed, dataset size 1982073
24072 games analyzed, dataset size 1982114
24073 games analyzed, dataset size 1982251
24074 games analyzed, dataset size 1982388
24075 games analyzed, dataset size 1982478
24076 games analyzed, dataset size 1982576
24077 games analyzed, dataset size 1982676
24078 games analyzed, dataset size 1982798
24079 games analyzed, dataset size 1982860
24080 games

24248 games analyzed, dataset size 1996781
24249 games analyzed, dataset size 1996867
24250 games analyzed, dataset size 1996978
24251 games analyzed, dataset size 1997054
24252 games analyzed, dataset size 1997155
24253 games analyzed, dataset size 1997189
24254 games analyzed, dataset size 1997239
24255 games analyzed, dataset size 1997310
24256 games analyzed, dataset size 1997414
24257 games analyzed, dataset size 1997505
24258 games analyzed, dataset size 1997564
24259 games analyzed, dataset size 1997607
24260 games analyzed, dataset size 1997670
24261 games analyzed, dataset size 1997872
24262 games analyzed, dataset size 1997979
24263 games analyzed, dataset size 1998012
24264 games analyzed, dataset size 1998041
24265 games analyzed, dataset size 1998080
24266 games analyzed, dataset size 1998201
24267 games analyzed, dataset size 1998259
24268 games analyzed, dataset size 1998314
24269 games analyzed, dataset size 1998427
24270 games analyzed, dataset size 1998479
24271 games

24439 games analyzed, dataset size 2011981
24440 games analyzed, dataset size 2012041
24441 games analyzed, dataset size 2012105
24442 games analyzed, dataset size 2012206
24443 games analyzed, dataset size 2012259
24444 games analyzed, dataset size 2012374
24445 games analyzed, dataset size 2012431
24446 games analyzed, dataset size 2012599
24447 games analyzed, dataset size 2012637
24448 games analyzed, dataset size 2012668
24449 games analyzed, dataset size 2012792
24450 games analyzed, dataset size 2012855
24451 games analyzed, dataset size 2012895
24452 games analyzed, dataset size 2012994
24453 games analyzed, dataset size 2013035
24454 games analyzed, dataset size 2013103
24455 games analyzed, dataset size 2013181
24456 games analyzed, dataset size 2013251
24457 games analyzed, dataset size 2013372
24458 games analyzed, dataset size 2013422
24459 games analyzed, dataset size 2013503
24460 games analyzed, dataset size 2013553
24461 games analyzed, dataset size 2013602
24462 games

24630 games analyzed, dataset size 2027838
24631 games analyzed, dataset size 2027920
24632 games analyzed, dataset size 2028030
24633 games analyzed, dataset size 2028079
24634 games analyzed, dataset size 2028128
24635 games analyzed, dataset size 2028232
24636 games analyzed, dataset size 2028275
24637 games analyzed, dataset size 2028420
24638 games analyzed, dataset size 2028504
24639 games analyzed, dataset size 2028617
24640 games analyzed, dataset size 2028637
24641 games analyzed, dataset size 2028746
24642 games analyzed, dataset size 2028811
24643 games analyzed, dataset size 2028812
24644 games analyzed, dataset size 2028839
24645 games analyzed, dataset size 2028893
24646 games analyzed, dataset size 2028948
24647 games analyzed, dataset size 2028996
24648 games analyzed, dataset size 2029070
24649 games analyzed, dataset size 2029135
24650 games analyzed, dataset size 2029199
24651 games analyzed, dataset size 2029287
24652 games analyzed, dataset size 2029346
24653 games

24821 games analyzed, dataset size 2043297
24822 games analyzed, dataset size 2043521
24823 games analyzed, dataset size 2043609
24824 games analyzed, dataset size 2043727
24825 games analyzed, dataset size 2043796
24826 games analyzed, dataset size 2043882
24827 games analyzed, dataset size 2043923
24828 games analyzed, dataset size 2044026
24829 games analyzed, dataset size 2044142
24830 games analyzed, dataset size 2044235
24831 games analyzed, dataset size 2044396
24832 games analyzed, dataset size 2044434
24833 games analyzed, dataset size 2044501
24834 games analyzed, dataset size 2044565
24835 games analyzed, dataset size 2044651
24836 games analyzed, dataset size 2044779
24837 games analyzed, dataset size 2044852
24838 games analyzed, dataset size 2044967
24839 games analyzed, dataset size 2045049
24840 games analyzed, dataset size 2045190
24841 games analyzed, dataset size 2045303
24842 games analyzed, dataset size 2045389
24843 games analyzed, dataset size 2045480
24844 games

In [6]:
X = np.dstack(X)
y = np.array(y, dtype='float32')
X = np.rollaxis(X, -1).reshape(-1, 12)

In [7]:
X.shape, y.shape

((2058384, 12), (2058384,))

In [8]:
print(bitboards_to_array(X[0]))

[[[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 1 0 0 0 0 1 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 1 0 0 1 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0

In [9]:
with h5py.File('positions.h5', 'w') as hf:
    hf.create_dataset("X", data=X, compression='gzip', compression_opts=5)
    hf.create_dataset("y", data=y, compression='gzip', compression_opts=5)

In [ ]:
print(max(y))

In [ ]:
sorted(y)[::-1][:50]

In [14]:
print(len(X))

2058384
